In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

Вам предоставляется набор данных, содержащий  заголовки статей с VC.RU, а также соответствующие им параметры, такие как количество просмотров и количество лайков. Ваша задача - обучить модель GPT-3.5 (update: можно взять любую opensource модель LLM) на этих заголовках+ выполнить fine-tuning модели, чтобы она могла генерировать заголовки по предоставленным данным новости,, которые привлекут большое количество просмотров и лайков (лайки и просмотры это таргет)
задача со звездочкой оценивать по заголовку кол-во потенциальных лайков и просмотров.


In [4]:
column_names = ["id", "seconds", "subsite", "url", "count_subs", "type", "author", "author_href", "count_subs1", "date", "href", "views", "hits", "likes", "comments", "title", "content", "content2"]
df = pd.read_csv('../data/dataset_vc.ru.csv', delimiter=';', names=column_names, header=0)
# df = df.iloc[1:] 

df.head(5)

,id,seconds,subsite,url,count_subs,type,author,author_href,count_subs1,date,href,views,hits,likes,comments,title,content,content2
0,890574,1698173845,Надя Сок,https://vc.ru/u/2496807-nadya-sok,0,1,NaN,NaN,NaN,24.10.2023 21:57:25 (Europe/Moscow),https://vc.ru/u/2496807-nadya-sok/890574-kak-u...,514,51,2,0.0,"Как улучшить жизнь, изменив мышление",Способность мыслить позитивно уже стало характ...,"Как улучшить жизнь, изменив мышление Способнос..."
1,890540,1698173493,Медиа,https://vc.ru/media,1013620,2,Таня Боброва,https://vc.ru/u/283507-tanya-bobrova,1921.0,24.10.2023 21:51:33 (Europe/Moscow),https://vc.ru/media/890540-otchet-spotify-komp...,11203,2771,10,59.0,Отчёт Spotify: компания получила первую кварта...,К концу года компания рассчитывает привлечь бо...,Отчёт Spotify: компания получила первую кварта...
2,886725,1698172976,U4i.Online,https://vc.ru/u4ionline,144,1,NaN,NaN,NaN,24.10.2023 21:42:56 (Europe/Moscow),https://vc.ru/u4ionline/886725-psihologichesko...,298,9,2,0.0,Психологическое консультирование: Обзор курса ...,NaN,Психологическое консультирование: Обзор курса ...
3,890511,1698171826,Арсений,https://vc.ru/u/2492053-arseniy,0,1,NaN,NaN,NaN,24.10.2023 21:23:46 (Europe/Moscow),https://vc.ru/u/2492053-arseniy/890511-ozon-be...,315,83,1,10.0,OZON: без невнимательного клиента и жизнь плоха,Суть: моя супруга по запарке добавила в корзин...,OZON: без невнимательного клиента и жизнь плох...
4,890525,1698171809,Екатерина Ревицкая Re:Digital,https://vc.ru/u/856974-ekaterina-revickaya-re-...,2,1,NaN,NaN,NaN,24.10.2023 21:23:29 (Europe/Moscow),https://vc.ru/u/856974-ekaterina-revickaya-re-...,545,106,2,4.0,Кейс: 3 700 заявок по 400 рублей из Яндекс Дир...,"Привет! С Вами рекламное агентство Re:Digital,...",Кейс: 3 700 заявок по 400 рублей из Яндекс Дир...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42684 entries, 0 to 42683
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           42684 non-null  int64  
 1   seconds      42684 non-null  int64  
 2   subsite      42684 non-null  object 
 3   url          42684 non-null  object 
 4   count_subs   42684 non-null  int64  
 5   type         42684 non-null  int64  
 6   author       28634 non-null  object 
 7   author_href  28634 non-null  object 
 8   count_subs1  28621 non-null  float64
 9   date         42684 non-null  object 
 10  href         42684 non-null  object 
 11  views        42684 non-null  int64  
 12  hits         42684 non-null  int64  
 13  likes        42684 non-null  int64  
 14  comments     42435 non-null  float64
 15  title        42219 non-null  object 
 16  content      33462 non-null  object 
 17  content2     42662 non-null  object 
dtypes: float64(2), int64(7), object(9)
memory usag

In [6]:
df.describe()

,id,seconds,count_subs,type,count_subs1,views,hits,likes,comments
count,42684.000000,4.268400e+04,4.268400e+04,42684.000000,28621.000000,42684.000000,42684.000000,42684.000000,42435.000000
mean,714129.540812,1.684974e+09,6.014324e+05,1.670532,415.075679,4456.557211,2278.624567,13.779144,25.403818
std,106337.745443,8.879633e+06,4.978305e+05,0.470025,1175.717638,7773.122981,6776.851969,33.288136,67.979902
min,77319.000000,1.666686e+09,-1.000000e+00,1.000000,-2.000000,0.000000,1.000000,0.000000,0.000000
25%,621699.250000,1.677692e+09,1.440000e+02,1.000000,7.000000,346.000000,114.000000,2.000000,0.000000
50%,716049.500000,1.685964e+09,9.975280e+05,2.000000,32.000000,997.000000,375.000000,4.000000,3.000000
75%,806109.250000,1.692870e+09,1.032483e+06,2.000000,213.000000,4869.250000,1712.000000,14.000000,15.000000
max,890574.000000,1.698174e+09,1.050733e+06,2.000000,21430.000000,99466.000000,394721.000000,1874.000000,1804.000000


In [17]:
# Drop the columns that are not required for future preprocessing
preproc_data = df.loc[:, ["id","subsite", "views", "likes", "title","content" ]]

preproc_data = preproc_data.dropna()

preproc_data = preproc_data.sort_values(by=['likes', 'views'])

preproc_data = preproc_data.iloc[46:]

# Display the first few rows of the modified DataFrame
preproc_data.head()

,id,subsite,views,likes,title,content
28033,656193,Zoringer,155,0,ВТБ и его убытки,Второй по активам банк в России ВТБ в 2022 год...
29281,644837,Будущее,176,0,В России научились прогнозировать нефтеотдачу ...,Специалисты Пермского национального исследоват...
27953,656908,Энди Кейдж,177,0,Ожидания VS статистика,"Когда делаешь творчество, а потом его продвига..."
32969,615203,skin pro,186,0,Гайд - стартануть на YouTube,"Подготовил для вас чек-лист, который можете ис..."
31385,626568,Диана Ряшенцева,205,0,Как «разделить» четвероногого члена семьи при ...,Домашние животные – это неделимое одушевленное...


In [8]:
preproc_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33032 entries, 0 to 42683
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       33032 non-null  int64 
 1   subsite  33032 non-null  object
 2   views    33032 non-null  int64 
 3   likes    33032 non-null  int64 
 4   title    33032 non-null  object
 5   content  33032 non-null  object
dtypes: int64(3), object(3)
memory usage: 1.8+ MB


In [18]:
preproc_data.to_csv("../data/preproc_data.csv")

In [19]:
sorted_df = preproc_data.sort_values(by=['likes', 'views'])

sorted_df.head(50)

,id,subsite,views,likes,title,content
28033,656193,Zoringer,155,0,ВТБ и его убытки,Второй по активам банк в России ВТБ в 2022 год...
29281,644837,Будущее,176,0,В России научились прогнозировать нефтеотдачу ...,Специалисты Пермского национального исследоват...
27953,656908,Энди Кейдж,177,0,Ожидания VS статистика,"Когда делаешь творчество, а потом его продвига..."
32969,615203,skin pro,186,0,Гайд - стартануть на YouTube,"Подготовил для вас чек-лист, который можете ис..."
31385,626568,Диана Ряшенцева,205,0,Как «разделить» четвероногого члена семьи при ...,Домашние животные – это неделимое одушевленное...
28126,655469,Александр,206,0,"Заблокировали аккаунт с тарифом, 160+ положите...",Почта привязанная к профилю - stupaksash@yande...
23946,696465,Энди Кейдж,216,0,О творческих рамках,"Пишешь себе, пишешь, а потом оказывается, что ..."
9506,816930,Евгений Адвокат Збинский,217,0,Потебительские каникулы,Обратиться за потребительскими кредитными кани...
11939,797323,Лида Дай Огня,217,0,Пост-Знакомство,"Привет, меня зовут Лида. Лида Дай Огня - Это П..."
9623,815878,Разовьёмся вместе?,230,0,Непринятый подарок. Притча,Всем привет! Я вернулась! Внедрили какие-нибуд...


In [20]:
# split df into train and test sets
train_df, test_df = train_test_split(preproc_data, test_size=0.2, random_state=42)

# split train_df into train and validation sets
train_df, val_df = train_test_split(preproc_data, test_size=0.2, random_state=42)

train_df.to_csv("../data/train_df.csv")
test_df.to_csv("../data/test_df.csv")
val_df.to_csv("../data/val_df.csv")